# Exploring the data

### Working with tabular data

In [20]:
import arcpy
import pandas as pd
import os

# Read the data and store it in a pandas dataframe, df.  
# A data frame is a 2-dimensional data structure.  
    
projectDir = r"C:\Users\echoi4\Desktop\esterArcGISProject-20230428T013544Z-001\esterArcGISProject" ## Replace with your data directory


try:
    # When working inside ArcGIS Pro,
    # use the project name "CURRENT"
    aprx = arcpy.mp.ArcGISProject("CURRENT")
except:
    # When working outside of ArcGIS Pro,
    # use the project full path file name.
    aprx = arcpy.mp.ArcGISProject(project_dir + "/estherArcGISProject.aprx")


workingDir = projectDir + "\output\\"
visDir = workingDir + "\pics\\"

# Create output directories if they don't already exist.
dirs = [workingDir, visDir]
for d in dirs:
    if not os.path.exists(d):
        os.mkdir(d)

myData = projectDir + '\esther.csv'
df = pd.read_csv(myData, header=5)

# Print the first 5 records to get a feel for the data.
df.head()

,Query Id,Query Name,Date,Title,Url,Domain,Sentiment,Page Type,Language,Country Code,Continent Code,Continent,Country,City Code,Account Type,Added,Assignment,Author,Avatar,Category Details,Checked,City,Display URLs,Expanded URLs,Facebook Author ID,Facebook Comments,Facebook Likes,Facebook Role,Facebook Shares,Facebook Subtype,Full Name,Full Text,Gender,Hashtags,Impact,Impressions,Instagram Comments,Instagram Followers,Instagram Following,Instagram Interactions Count,...,Twitter Channel Role,Twitter Followers,Twitter Following,Twitter Reply Count,Twitter Reply to,Twitter Retweet of,Twitter Retweets,Twitter Tweets,Twitter Verified,Updated,Reach (new),Blog Name,Copyright,Engagement Type,Entity Info,Item Review,Linkedin Comments,Linkedin Engagement,Linkedin Impressions,Linkedin Likes,Linkedin Shares,Linkedin Sponsored,Linkedin Video Views,Page Type Name,Parent Blog Name,Parent Post Id,Pub Type,Publisher Sub Type,Rating,Reddit Author Awardee Karma,Reddit Author Awarder Karma,Reddit Author Karma,Reddit Score,Region,Region Code,Root Blog Name,Root Post Id,Subreddit,Subreddit Subscribers,Weblog Title
0,2000705381,LT - SLF Twitter Penn since 2011,2022-01-26 22:20:25.0,“It’s a micro mini pig! It fits in a teacup!” ...,https://www.reddit.com/r/TeenMomOGandTeenMom2/...,reddit.com,neutral,reddit,en,NaN,NaN,NaN,NaN,NaN,NaN,2022-02-14T17:53:04.954+0000,NaN,Midnight_Ravioli,https://styles.redditmedia.com/t5_zc3na/styles...,"{id=11239953, name=Other, displayName=null, pa...",False,NaN,NaN,NaN,NaN,0,0,NaN,0,NaN,Midnight_Ravioli,Reminds me of Ester the wonder pig. My mom bou...,unknown,NaN,NaN,0,0,0,0,0,...,NaN,0,0,0,NaN,NaN,0,0,False,2022-02-14T17:53:04.954+0000,NaN,NaN,NaN,NaN,"{entityId=13417213, entityConfidence=HIGH, url...",NaN,0.0,0.0,0,0,0,false,0,Reddit,NaN,sda395,BW_REDDIT_REPLY,NaN,NaN,0.0,23.0,13666.0,11,NaN,NaN,NaN,sda395,TeenMomOGandTeenMom2,108360,Midnight_Ravioli
1,2000705381,LT - SLF Twitter Penn since 2011,2022-01-10 01:00:58.0,"@mkramer For giggles, you can follow Ester the...",http://twitter.com/MarySwannParry/statuses/148...,twitter.com,neutral,twitter,en,USA,NORTH AMERICA,North America,United States of America,NaN,individual,2022-02-14T17:53:03.014+0000,NaN,MarySwannParry,https://audiences.brandwatch.com/api/audiences...,"{id=11239953, name=Other, displayName=null, pa...",False,NaN,NaN,https://www.instagram.com/estherthewonderpig/?...,NaN,0,0,NaN,0,NaN,MarySwannParry (Mary Swann Parry),"@mkramer For giggles, you can follow Ester the...",female,NaN,0.2,1684,0,0,0,0,...,NaN,1684,2712,0,http://twitter.com/mkramer/statuses/1480193116...,NaN,0,10978,False,2022-02-14T17:53:03.014+0000,901.0,NaN,NaN,REPLY,"{entityId=209330, entityConfidence=HIGH, url=h...",NaN,0.0,0.0,0,0,0,false,0,Twitter,NaN,NaN,Twitstream,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MarySwannParry (Mary Swann Parry)
2,2000705381,LT - SLF Twitter Penn since 2011,2021-11-20 17:45:39.0,@PixieZelda Ester the Wonder Pig would approve!,http://twitter.com/MiralukaFan/statuses/146211...,twitter.com,neutral,twitter,en,AUS,OCEANIA,Australia/Oceania,Australia,NaN,individual,2022-02-14T17:53:05.447+0000,NaN,MiralukaFan,https://audiences.brandwatch.com/api/audiences...,"{id=11239953, name=Other, displayName=null, pa...",False,NaN,NaN,NaN,NaN,0,0,NaN,0,NaN,MiralukaFan (MiralukaFan),@PixieZelda Ester the Wonder Pig would approve!,unknown,NaN,0.0,156,0,0,0,0,...,NaN,156,3886,0,http://twitter.com/PixieZelda/statuses/1462099...,NaN,0,14367,False,2022-02-14T17:53:05.447+0000,0.0,NaN,NaN,REPLY,"{entityId=10798, entityConfidence=MEDIUM, url=...",NaN,0.0,0.0,0,0,0,false,0,Twitter,NaN,NaN,Twitstream,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MiralukaFan (MiralukaFan)
3,2000705381,LT - SLF Twitter Penn since 2011,2021-11-08 10:37:02.0,@angelacrazygirl @xo415 @AdamsCumbie The probl...,http://twitter.com/MissanMissen/statuses/14576...,twitter.com,positive,twitter,en,NaN,NaN,NaN,NaN,NaN,individual,2022-02-14T17:53:06.016+0000,NaN,MissanMissen,https://audiences.brandwatch.com/api/

In [2]:
# What fields does the data have?
df.columns

Index(['Query Id', 'Query Name', 'Date', 'Title', 'Url', 'Domain', 'Sentiment',
       'Page Type', 'Language', 'Country Code',
       ...
       'Reddit Author Awarder Karma', 'Reddit Author Karma', 'Reddit Score',
       'Region', 'Region Code', 'Root Blog Name', 'Root Post Id', 'Subreddit',
       'Subreddit Subscribers', 'Weblog Title'],
      dtype='object', length=105)

In [3]:
# Print ALL the field names.
for columnName in df.columns:
    print(columnName,end='        ',)

Query Id        Query Name        Date        Title        Url        Domain        Sentiment        Page Type        Language        Country Code        Continent Code        Continent        Country        City Code        Account Type        Added        Assignment        Author        Avatar        Category Details        Checked        City        Display URLs        Expanded URLs        Facebook Author ID        Facebook Comments        Facebook Likes        Facebook Role        Facebook Shares        Facebook Subtype        Full Name        Full Text        Gender        Hashtags        Impact        Impressions        Instagram Comments        Instagram Followers        Instagram Following        Instagram Interactions Count        Instagram Likes        Instagram Posts        Interest        Last Assignment Date        Latitude        Location Name        Longitude        Media Filter        Media URLs        Mentioned Authors        Original Url        Priority        Profess

In [4]:
# How many columns and rows does the data have?
df.shape

(1121, 105)

In [5]:
# Which countries are there in the "Country" field?
# The Series method named unique returns a Numpy array containing the unique values in a series. 
df['Country'].unique()

array([nan, 'United States of America', 'Australia', 'Canada',
       'Netherlands', 'United Kingdom', 'Poland', 'Japan', 'New Zealand',
       'Italy', 'Vietnam', 'Republic of Ireland', 'Finland', 'Croatia',
       'Israel', 'Germany', 'The Bahamas', 'Brazil', 'Russia', 'Norway',
       'Indonesia', 'South Africa', 'Pakistan', 'India',
       'Republic of Serbia', 'Argentina', 'Singapore', 'Sweden',
       'Colombia', 'Mexico', 'Gibraltar', 'Venezuela'], dtype=object)

In [6]:
# Change the name in the Country column for USA to match GIS data name for USA
df['Country'] = df['Country'].replace('United States of America','United States')
df['Country'].unique()

array([nan, 'United States', 'Australia', 'Canada', 'Netherlands',
       'United Kingdom', 'Poland', 'Japan', 'New Zealand', 'Italy',
       'Vietnam', 'Republic of Ireland', 'Finland', 'Croatia', 'Israel',
       'Germany', 'The Bahamas', 'Brazil', 'Russia', 'Norway',
       'Indonesia', 'South Africa', 'Pakistan', 'India',
       'Republic of Serbia', 'Argentina', 'Singapore', 'Sweden',
       'Colombia', 'Mexico', 'Gibraltar', 'Venezuela'], dtype=object)

In [7]:
# How many countries is that?
# Use the built-in len method to find the length of the array of unique country names.
print(len(df['Country'].unique()))

32


In [8]:
# The Page Type column indicates the detected data source (e.g., reddit, twitter, news,...)
## Add code to determine which page types are there in the "Page Type" field.
df['Page Type'].unique()

array(['reddit', 'twitter', 'news', 'forum', 'review', 'tumblr', 'blog',
       'youtube', 'instagram', 'facebook'], dtype=object)

In [9]:
# How many page types are there?
## Add code to print the number of unique page types.
print(len(df['Page Type'].unique()))

10


### Exploring text content 
Suppose we want to search for references to vegans or happiness in the Tweets.  We can use regular expressions to search for key words.  A _regular expression_ is a sequence of characters that specifies a search pattern in text (It's a very deep topic. This provides a gentle introduction: [How to use Regex in Pandas](https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/)).  The code below has a simple regular expression that uses | to mean 'or':



In [10]:
regular_expression = 'happy|vegan|happiness|joy|smile'

# Get a dataframe of records in which the content of the Full Text column has any of these terms. 

# For the moment, we only care about the column that contains the full text of the posts.
# To make a series from a particular columns, you can specify the column name in square braces
ft_series = df['Full Text']
print(ft_series.size)

# Prepare a dataframe that only has the non-null Tweets 
ft_series = ft_series.dropna()
print(ft_series.size)


# str.contains returns a Boolean series.  A series is a one dimensional data structure, like an array.
#boolean_series = df2['Full Text'].str.contains(regular_expression, case=False)
boolean_series = ft_series.str.contains(regular_expression, case=False)

# loc is a dataframe property that allows you to access a group of rows and/or columns 
# based either on labels or a boolean array.
# Here we're using it with the Boolean series to get the Tweets that match our terms 
# (the ones where str.contains returned True).
joy_series = ft_series.loc[boolean_series]

joy_series= joy_series.drop_duplicates()
print(joy_series.size)
joy_series.head()


1121
1120
240


16    Books that teach compassion & kindness to chil...
19    RT @MichelleM41811 So, so, soo happy to be bac...
34    So, so, soo happy to be back working out with ...
36    To me, chickens hurt my heart the most. But I'...
38    Love the sweet, contented "smile" on Ester the...
Name: Full Text, dtype: object

### Extract date time info. (year, month, day, or time) with the Python datetime module.

In [11]:
import datetime
theDate = '06/14/2015  4:11:11 PM'
# Create a string that represents the format of the date.  
# The format code tells it where to look for each element. 
# For example, %m represents *month*, whereas %M represents *minute*.
# A full table of format codes can be found here:  (Python strptime)[https://www.programiz.com/python-programming/datetime/strptime]
# The datetime module's datetime class method named *strptime* stands for string parsed time.
datem = datetime. datetime. strptime(theDate, '%m/%d/%Y %H:%M:%S %p')
print(datem.year)

2015


### Working with dates in Pandas

Dataframes have a DatetimeIndex class that makes it easy to extract the parts of a date or time from a date time stamp.

In [12]:
# So many columns... does it have a Date column?
"Date" in df.columns

True

In [13]:
# How about a Year column?
"Year" in df.columns

False

In [21]:
# Extract Year from the Date column and make new columns for these.
df['Year'] = pd.DatetimeIndex(df['Date']).year
df.columns

Index(['Query Id', 'Query Name', 'Date', 'Title', 'Url', 'Domain', 'Sentiment',
       'Page Type', 'Language', 'Country Code',
       ...
       'Reddit Author Karma', 'Reddit Score', 'Region', 'Region Code',
       'Root Blog Name', 'Root Post Id', 'Subreddit', 'Subreddit Subscribers',
       'Weblog Title', 'Year'],
      dtype='object', length=106)

In [22]:
## Write code here to add a Month column to the dataframe (df)
df['Month'] = pd.DatetimeIndex(df['Date']).month

In [23]:
# Let's check which years are included in the data. 
# The Series method named unique returns a Numpy array containing the unique values in a series. 
df['Year'].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012],
      dtype=int64)

In [24]:
# Now we have years!  Let's find out how many Canadian posts there were in 2021.
# To select the 2021 Canadian posts, we need to select rows based on two column conditions. 
#df_special = df[(df['Year'] == 2021) & (df['Country']== 'Canada')]

boolean_series = (df['Year'] == 2019) & (df['Country']== 'Canada')
df_CA_21 = df[boolean_series]
df_CA_21.shape

(18, 107)

### Select columns / rows to a csv to be joined with the country boundaries.

In [25]:
# Select pertinent columns.
df_small = df[['Page Type','Country', 'Year', 'Full Text']]  ## Add the month column 

# Print the number of rows and columns.
print(df_small.shape)

# Remove rows in which any of the values are empty.
df_small = df_small.dropna()

print(df_small.shape)

smallCSV = workingDir + 'esther_reduced.csv'
df_small.to_csv(smallCSV)

(1121, 4)
(490, 4)


In [26]:
# Explore the reduced dataset.
# The value_counts method provides a frequency count of values in a field.  
# As you can see below, 2016 was a big year for Esther the Pig. 
print(df_small['Year'].value_counts())

2016    100
2017     98
2018     68
2015     62
2014     60
2019     52
2020     28
2021     18
2013      3
2022      1
Name: Year, dtype: int64


In [27]:
# Explore the reduced dataset.
# Where is Esther most discussed?
## Add code to print the frequency count for Country.  
# As you can see below, 2016 was a big year for Esther the Pig. 
print(df_small['Year'].value_counts())

2016    100
2017     98
2018     68
2015     62
2014     60
2019     52
2020     28
2021     18
2013      3
2022      1
Name: Year, dtype: int64


In [28]:
# Explore the reduced dataset.
# Which countries have postings each year? 
for val in sorted(df_small['Year'].unique()):
    # selecting rows based on condition
    rslt_df = df_small[df_small['Year'] == val]
    print(f'Year:   {val}')
    print(rslt_df['Country'].unique())

Year:   2013
['Australia' 'United States of America']
Year:   2014
['United States of America' 'United Kingdom' 'Sweden' 'Canada' 'India'
 'Colombia' 'Mexico' 'Gibraltar' 'Venezuela']
Year:   2015
['United States of America' 'Australia' 'United Kingdom' 'Canada' 'Italy'
 'Sweden']
Year:   2016
['Republic of Ireland' 'United Kingdom' 'Australia'
 'United States of America' 'Italy' 'India' 'Canada' 'South Africa'
 'Republic of Serbia' 'Finland' 'Germany' 'Croatia' 'New Zealand'
 'Argentina' 'Singapore']
Year:   2017
['United States of America' 'Australia' 'Canada' 'Israel' 'Brazil'
 'Russia' 'United Kingdom' 'Norway' 'Indonesia' 'South Africa' 'Pakistan']
Year:   2018
['United States of America' 'Canada' 'Croatia' 'Australia'
 'United Kingdom' 'Japan' 'Israel' 'Republic of Ireland' 'Germany'
 'The Bahamas']
Year:   2019
['United States of America' 'Italy' 'Canada' 'Japan' 'Vietnam'
 'United Kingdom' 'Republic of Ireland' 'Australia' 'Finland']
Year:   2020
['Canada' 'United States of Ame

### Make a copy of the worldwide countries dataset

Other data will be joined to this, so we want to preserve the original before modifying the table.

Data source: 
[ArcGIS Hub World Countries Generalized] (https://hub.arcgis.com/datasets/2b93b06dc0dc4e809d3c8db5cb96ba69_0/explore?location=-0.101905%2C0.000000%2C2.09)

In [29]:
import arcpy
arcpy.env.workspace = projectDir + r'esterArcGISProject.gdb'
arcpy.env.overwriteOutput = True
worldData = 'world_countries'
worldDataCopy = workingDir + 'worldEsther.shp'
arcpy.CopyFeatures_management(worldData, worldDataCopy)

<Result 'C:\\Users\\echoi4\\Desktop\\esterArcGISProject-20230428T013544Z-001\\esterArcGISProject\\output\\worldEsther.shp'>

In [31]:
# Get a the base name (without the extention) of the input data.  We'll use this to name files in the next part.
baseName = os.path.basename(myData)
baseName = os.path.splitext(baseName)[0]
baseName

'esther'

### Create a table for each time step and join the tables with the GIS data

In [32]:
# Create an empty dictionary to store the number of records per year.
yearly_count = {}

# For each year in the reduced dataset, create a csv file.

for yr in sorted(df_small['Year'].unique()):
    # Select rows based on the year
    rslt_df = df_small[df_small['Year'] == yr]
    print(f"Number of records in {yr}: {len(rslt_df)}")
    yearly_count[yr] = len(rslt_df)

    # Prepare an output name
    annualCSV = workingDir + f'{baseName}{yr}.csv'
    
    # Write the data frame to csv
    rslt_df.to_csv(annualCSV)
    
    print(f'Year:   {yr}')
    print(rslt_df['Country'].unique())
    
    # Join the data
    joinedData = arcpy.management.AddJoin(
        in_layer_or_view = worldDataCopy, 
        in_field = 'COUNTRY', 
        join_table = annualCSV, 
        join_field = 'Country', 
    join_type = 'KEEP_COMMON')
    
    curData = workingDir + f'{baseName}{yr}.shp'
    try:
        arcpy.CopyFeatures_management(joinedData, curData)
    except:
        print("File already exists.  New copy not created.")

Number of records in 2013: 3
Year:   2013
['Australia' 'United States of America']
Number of records in 2014: 60
Year:   2014
['United States of America' 'United Kingdom' 'Sweden' 'Canada' 'India'
 'Colombia' 'Mexico' 'Gibraltar' 'Venezuela']
Number of records in 2015: 62
Year:   2015
['United States of America' 'Australia' 'United Kingdom' 'Canada' 'Italy'
 'Sweden']
Number of records in 2016: 100
Year:   2016
['Republic of Ireland' 'United Kingdom' 'Australia'
 'United States of America' 'Italy' 'India' 'Canada' 'South Africa'
 'Republic of Serbia' 'Finland' 'Germany' 'Croatia' 'New Zealand'
 'Argentina' 'Singapore']
Number of records in 2017: 98
Year:   2017
['United States of America' 'Australia' 'Canada' 'Israel' 'Brazil'
 'Russia' 'United Kingdom' 'Norway' 'Indonesia' 'South Africa' 'Pakistan']
Number of records in 2018: 68
Year:   2018
['United States of America' 'Canada' 'Croatia' 'Australia'
 'United Kingdom' 'Japan' 'Israel' 'Republic of Ireland' 'Germany'
 'The Bahamas']
Num

#### Remove layers that have been added to the map during processing
This will allow us to control which layers are visible for screenshot capturing.
The pictures could have been captured while the data were being created, but
these steps are separated to make them easier to distinguish.

In [33]:
# aprx is the ArcGIS project object

# Get a list of maps in this project.
myMap = aprx.listMaps('Map')[0]
layers = myMap.listLayers()
layer_object = layers[0]
count = 0 
while "Worldwide" not in layer_object.name and count < len(layers):
    print(layer_object.name)
    myMap.removeLayer(layer_object)
    count = count + 1
    layer_object = layers[count]
    
for i in layers:
    print (i.name)

esther2022
worldEsther_Layer9
esther2021
worldEsther_Layer8
esther2020
worldEsther_Layer7
esther2019
worldEsther_Layer6
esther2018
worldEsther_Layer5
esther2017
worldEsther_Layer4
esther2016
worldEsther_Layer3
esther2015
worldEsther_Layer2
esther2014
worldEsther_Layer1
esther2013
worldEsther_Layer
worldEsther
Moth2021


AttributeError: The attribute 'name' is not supported on this instance of Layer.

### Create map layers and take screenshots

We'll do this by looping through the years. Earlier, we created a shapefile for each year.  Now we'll make a layer for each shapefile.   We'll add it to the map, capture a screenshot (a png image), and turn then that layer's visibility off, so that we can do the same again with the next shapefile.

In [34]:
# aprx is the ArcGIS project object

# Get a list of maps in this project.
myMap = aprx.listMaps('Map')[0]

custom_layout = aprx.listLayouts("*surround*")[0]
for elem in custom_layout.listElements():
    print(elem.name)
    if "Text" in elem.name:
        textbox = elem


RGBs = [[190, 210, 255, 60], [245, 122, 122, 60], [255, 255, 0, 60]]

# For each year in the data, add a layer and export an image of it.
for yr in sorted(df_small['Year'].unique()):

    # Get the name of the shapefile associated with the current year. 
    curData = workingDir + f'{baseName}{yr}.shp'
    
    # Add the shapefile as a layer on the map.
    myMap.addDataFromPath(curData)
    # Set the text string to current year
    textbox.text = yr
    
    # Now we need the layer object associated with our new layer. 
    # This gives access to the symbology properties of the layer.  
    # Get a list of layor objects for the layers on this map.
    layerObjects = myMap.listLayers()
    
    # Get the first one (the most recently added to the map)
    lay = layerObjects[0]

    print(lay.name)
    
    sym = lay.symbology
    
    # Set an RGB color for the layer.  
    # Provide the colors as a list of numbers (red, green, and blue, plus alpha for transparency).
    # E.g., [190, 210, 255, 60] is a purple with opacity of 60%.
    theColor = [245, 122, 122, 100]  # pink with no transparency
    sym.renderer.symbol.color = {'RGB': theColor}  

    ## As an alternative to specifying RGB values, you can apply symbology from the gallery.
    # Try something you like.  See the named_symbology_examples.png for other names 
    # or look at the choices in ArcGIS Pro to see more.
    # sym.renderer.symbol.applySymbolFromGallery('Mangrove')

    ## Experiment with the outline by uncommenting the code below
    # sym.renderer.symbol.outlineColor = {'CMYK' : [25, 50, 75, 25, 100]}
    lay.symbology = sym
    #if theMonth == 12:
    #    myMap.defaultView.zoomToAllLayers()
    print ('{}:{}'.format(lay.name, lay.visible))
    outPic = visDir+f'{baseName}{str(yr).zfill(2)}.png'
    #myMap.defaultView.exportToPNG(outPic, width=800, height=600)
    custom_layout.exportToPNG(outPic)
    lay.visible = False
    
    del lay
del myMap
del aprx


Legend
Text
North Arrow
Map Frame
esther2013
esther2013:True
esther2014
esther2014:True
esther2015
esther2015:True
esther2016
esther2016:True
esther2017
esther2017:True
esther2018
esther2018:True
esther2019
esther2019:True
esther2020
esther2020:True
esther2021
esther2021:True
esther2022
esther2022:True


### Creat an animation

In [39]:
import imageio
images = []
import os
mapPics = os.listdir(visDir)

movieName = visDir + '/movie.gif'
if os.path.exists(movieName):
    os.remove(movieName)

for pic in mapPics:
    if pic.endswith("png"):
        images.append(imageio.imread(visDir + pic))


imageio.mimsave(movieName, images, duration=0.8)
image_count = len(images)
print('{} created using {} maps!'.format( movieName, image_count))


C:\Users\echoi4\Desktop\esterArcGISProject-20230428T013544Z-001\esterArcGISProject\output\\pics\/movie.gif created using 10 maps!


In [40]:
print(mapPics)

['esther2013.png', 'esther2014.png', 'esther2015.png', 'esther2016.png', 'esther2017.png', 'esther2018.png', 'esther2019.png', 'esther2020.png', 'esther2021.png', 'esther2022.png', 'Moth.html']


### Create an html display

In [41]:
header = """<!DOCTYPE html>
<html>
<head>
<style>
div.gallery {
  border: 1px solid #ccc;
}

div.gallery:hover {
  border: 1px solid #777;
}

div.gallery img {
  width: 100%;
  height: auto;
}

div.desc {
  padding: 15px;
  text-align: center;
}

* {
  box-sizing: border-box;
}

.responsive {
  padding: 0 6px;
  float: left;
  width: 24.99999%;
}

@media only screen and (max-width: 700px) {
  .responsive {
    width: 49.99999%;
    margin: 6px 0;
  }
}

@media only screen and (max-width: 500px) {
  .responsive {
    width: 100%;
  }
}

.clearfix:after {
  content: "";
  display: table;
  clear: both;
}
</style>
</head>
<body>
""" 

bodyContent = f"""<h2>{baseName.title()} Posts Over {image_count} Years </h2>
<h4>Author: Grace Choi</h4>
<h4>Data source: {baseName} Esther the Wonder Pig</h4>"""

mapPics = os.listdir(visDir)

mapPics = [picName for picName in mapPics if picName.endswith('.png') ]

for pic in mapPics:
    dateVal = os.path.splitext(pic)[0]
    dateVal = dateVal.replace(baseName,'')
    dateVal = int(dateVal)
    bodyContent = bodyContent + f"""<div class="responsive"><div class="gallery">
  <a target="_blank" href={pic}>
    <img src={pic} alt="{os.path.splitext(pic)[0]}" width="800" height="600">
  </a>
  <div class="desc">{yearly_count[dateVal]} {baseName.title()} post(s) in {dateVal}</div>
</div></div>"""

# Add movie
movieName = os.path.basename(movieName)
bodyContent = bodyContent + f"""<div class="responsive"><div class="gallery">
  <a target="_blank" href={movieName}>
    <img src={movieName} alt="animation" width="800" height="600">
  </a>
  <div class="desc">{baseName.title()} posts over time.</div>
</div></div>"""


footer = """</body>
</html>"""

html_file_name = visDir + f'{baseName}.html'

with open( html_file_name,'w') as outFileObject:
  outFileObject.write(header)
  outFileObject.write(bodyContent)
  outFileObject.write(footer)
os.startfile(html_file_name)